In [ ]:

import os
import time
import numpy as np
import nibabel as nib
import pandas as pd
from tqdm import tqdm
from gen_slice import gen_slice, rescale,time_elapsed, gen_slice_masked,scale_xy
from gen_slice import nii_3Dto2D, flip_rotate
from gen_slice import gen_txt,gen_slice_ROI,make_ROI,gen_slice_ROI_from_parquet,make_ROI_repeat


Reconstruct nifti images from raw txt, bin = 50 (marmoset) or 100 (macaque) or 10 (mouse)

In [5]:
path_raw = r'txt_data_path/'
files_txt = [f for f in os.listdir(path_raw) if 'txt' in f]
files_txt.sort()
files_id = [f.split('.')[0] for f in files_txt]

In [6]:
path_sagittal_slices = r'txt_data_to_nifti_path/'
if not os.path.exists(path_sagittal_slices):
    os.makedirs(path_sagittal_slices)

In [ ]:
for f in files_txt:
    path_input = os.path.join(path_raw, f)
    path_output = os.path.join(path_sagittal_slices, '.'.join((f, 'nii.gz')))
    gen_slice(path_input, path_output, 50)

rescale the size of nifti images to the same size (1000x1000 )

In [ ]:
path_origin = 'txt_data_to_nifti_path/'
path_large_slice = [os.path.join(path_origin, 'large_' + f) for f in os.listdir(path_origin)]
path_origin_slice = [os.path.join(path_origin, f) for f in os.listdir(path_origin)]
for path_in, path_out in tqdm(zip(path_origin_slice, path_large_slice)):
    rescale(path_in, path_out,1000)

use the '2D_register' in the Linux console to register each slice to same space

 generate 2d nifti slice from 3d nifti volume totalRNA or mask or ROI

In [ ]:
path_nii_volume = 'macaque_ceb.nii.gz'
path_nii_slice = 'macaque_ceb_slice.nii.gz'

nii_3Dto2D(path_nii_volume, path_nii_slice,slice_dim=1, prefix='coronal')


manually rotate slice to the nearest angle between raw data slice and the current slice from 3d 

In [ ]:
path_sagittal_reg = './coronal_slice_reg/'
path_sagittal_rotate = './coronal_slice_rotatereturn'
sagittal_slices = [f for f in os.listdir(path_sagittal_reg) if 'coronal' in f]
mask_slices = [f for f in os.listdir(path_sagittal_reg) if 'ROI' in f]
sagittal_slices.sort(), mask_slices.sort()

In [9]:
rotate_list = [
    0, 2, 1, 0, 2, 0, 1, 0, 0, 3,
    0, 3, 3, 0, 3, 0, 2, 0, 0, 0,
    0, 0, 1, 0, 0, 0, 0, 0, 2, 2,
    3, 2, 0, 2, 0, 3, 0, 3, 0, 2,
    0, 3, 2, 3, 0, 0, 2, 2, 3, 0,
    0, 0, 3, 0, 0, 2, 2, 0, 3, 1,
    3, 3, 1, 2, 3, 3, 1, 2, 3, 2,
    0, 2, 1, 3, 2, 3, 0, 2, 3, 3,
    1, 1, 3, 3, 3, 3, 1, 0, 0, 1,
    0, 3, 2, 0, 2, 0, 3, 2, 1, 3,
    2, 1, 2, 3, 2, 0, 3, 0, 2, 2,
    3, 1, 1, 0, 3, 1]

In [10]:
for k, sagittal, mask in zip(rotate_list, sagittal_slices, mask_slices):
    #flip_rotate(os.path.join(path_sagittal_reg, sagittal), os.path.join(path_sagittal_rotate, 'rotate_' + sagittal), k)
    flip_rotate(os.path.join(path_sagittal_reg, mask), os.path.join(path_sagittal_rotate, 'rotate_' + mask), k)

Registraion

Registraion the rotatereturn totalNRA and mask to the nii-slice(large-1600) from the txt to genera the nii mask 